# Ryder: Global Expansion Research #
<a id="top" />

This Jupyter notebook contains the summaries and visualizations that should accompany the report on Ryder's company expansion into other countries, based on key indicators derived from a country's level of crime and stability, basic measures of health and wealth, and ratios of public education spending.

***

## Table of Contents ##

1. [Title](#top)
2. [Introduction](#introduction)
3. [Dependencies](#dependencies)
4. [Prepare for Analysis](#prepare)<br/>
    a. [Import Countries](#import_countries)<br/>
    b. [Selecting Countries](#select_countries)
    c. [Import Data](#import_data)
        i. [Import MFI](#import_data_mfi)
        ii. [Import GTD](#import_data_gtd)
        iii. [Import PED](#import_data_ped)
5. [Queries](#queries)

***

<a id="introduction" />

## Introduction ##

This notebook contains queries made against cleansed versions of the following Kaggle datasets:

- [Global Terrorism Database](https://www.kaggle.com/START-UMD/gtd) (2018)
- [Infant Mortality, Fertility, Income per Capita](https://www.kaggle.com/burhanykiyakoglu/infant-mortality-fertility-income) (2018)
- [Public Education Expenditure as share of GDP](https://www.kaggle.com/ibrahimmukherjee/gdp-world-bank-data#public-education-expenditure-as-share-of-gdp.csv) (2018)

Datasets were prepared using a combination of different commandline tools, including `bash`, `awk`, `sed`, and `python`. The original wide datasets were broken up to better facilitate querying. Details on that process are included in the accompanying documentation.

***

<a id="dependencies" />

## Dependencies ##

This notebook requires the following packages:

    - numpy
    - pandas 
    - matplotlib

Before we can plot information and fiddle with the data, we need to import these modules.


In [ ]:
import numpy as np         # numpy library for vectorized computations.
import pandas as pd        # pandas library for manipulating dataframes.

### pandas & numpy ###

Pandas allows us to manipulate dataframes.

We can create these dataframes by reading in our data. `*.tsv` files can be imported using the `read_csv()` function. Note the custom `\t` separator used with tab-separated-value files.

```python
# Outputs a dataframe from a parsed data file.
dataframe = pd.read_csv("<filename>.tsv", sep="\t")
```

### matplotlib ###

We can also import matplotlib and the pylot modules for configuration and later use.

In [ ]:
import matplotlib as mpl   # matplotlib library for global settings.
import matplotlib.pyplot as plt  # Our plotting functions from matplotlib.

In [ ]:
display(plt.style.available)  # Display what styles are available.

In [ ]:
# %matplotlib inline
%config InlineBackend.figure_format = 'png'
                                     # Above lines makes plots appear as inline svgs.
mpl.rcParams['figure.dpi'] = 100     # Apply DPI to matplotlib inline plots.
plt.style.use(['fivethirtyeight', 'seaborn-dark', 'ggplot'])     # Apply particular styles.
plt.plot(np.sin(np.linspace(0, 2 * np.pi)), 'r-o')  # Make the plot.
plt.show()  # Show the plot.

***

<a id="prepare"></a>

## Preparing for Analysis ##

The following packages contain support functions unique to this particular report.

The `analysis.analyser` package contains the `analyser` and `country` modules that do a bulk of the computational work. The `analysis.utils` package contains utility modules used to do repetitive tasks across the entire project.

In [ ]:
from analysis.analyser import analyser
from analysis.analyser.country import Country
from analysis.utils import parser
from analysis.utils import validate

<a id="import_countries" />

#### Selecting Candidate Countries ####

We need to select countries from our dataset's available countries. This means finding the above three (3) countries and ensuring they have entries across all three datasets.

In [ ]:
# Outputs a dataframe from a parsed data file.
countries_df = parser.read_tsv("../data/country_codes.tsv")
display(countries_df)

In [ ]:
# Get the np.array of unique countries that appear in the public education expenditure dataset.
ped_countries_df = parser.read_tsv("../data/ped/ped_countries.tsv")
keys_ped = Country.from_frame(countries_df, search=list(ped_countries_df.iloc[:,0].unique()))
print(Country.format(keys_ped, sep="\n"))

In [ ]:
# Get np.array of unique countries that appear in the infant mortality, fertility, income per capita dataset.
mfi_countries_df = parser.read_tsv("../data/mfi/mfi_countries.tsv")
keys_mfi = Country.from_frame(countries_df, search=mfi_countries_df.iloc[:,0].unique())
print(Country.format(keys_mfi[:5], sep="\n"), "\n...\n", Country.format(keys_mfi[-5:], sep="\n"))

In [ ]:
# Get np.array of unique countries that appear in the global terrorism database.
gtd_countries_df = parser.read_tsv("../data/gtd/gtd_countries.tsv")
keys_gtd = Country.from_frame(countries_df, search=gtd_countries_df.iloc[:,0].unique())
print(Country.format(keys_gtd[:5], sep="\n"), "\n...\n", Country.format(keys_gtd[-5:], sep="\n"))

In [ ]:
# Compute the unique available countries among the datasets.
country_codes = {
    "ped": list(map(lambda country: country.code, keys_ped)),
    "mfi": list(map(lambda country: country.code, keys_mfi)),
    "gtd": list(map(lambda country: country.code, keys_gtd))
}

# Find the available countries.
unique_codes = analyser.find_intersection(*country_codes.values())
print(f'Unique Codes: {unique_codes}')

available_countries = countries_df[countries_df["Code"].isin(unique_codes)]
display(available_countries)

In [ ]:
# Clear unused variables in IPython.
%reset_selective -f "^id$"
%reset_selective -f "^code$"
%reset_selective -f "^name$"

# Equivalent: del keys_ped
# Equivalent: del keys_mfi
# Equivalent: del keys_gtd
%reset_selective -f "_ped$"
%reset_selective -f "_mfi$"
%reset_selective -f "_gtd$"

# Equivalent: del ped_countries_df
# Equivalent: del mfi_countries_df
# Equivalent: del gtd_countries_df
%reset_selective -f "_countries_df$"

# Equivalent: del country_codes
# Equivalent: del unique_codes
%reset_selective -f "_codes$"

<a id="select_countries" />

#### Selecting Countries ####

Now, with a sense of what countries are available, we can select 3 countries (besides the current one, USA) for comparison.

In [ ]:
# The countries we want to evaluate.
codes = [
    "GBR",  # United Kingdom,
    "JPN",  # Japan,
    "SWE",  # Sweden,
    "USA"   # United States
    ]
display(codes)

In [ ]:
# Select these countries from the set of available countries.
selected_df = available_countries[available_countries["Code"].isin(codes)]
selected_df = selected_df.set_index("Code", drop=False)
display(selected_df)

In [ ]:
# Convert selected countries into Country representations for use across all datasets.
selected_countries = Country.get_countries(selected_df)
print(Country.format(selected_countries, sep="\n"))

<a id="import_data" />

### Importing Data ###

In order to query the data, we need to form our `pandas.DataFrame` representations.

<a id="import_data_mfi" />

#### Mortality ####

The mortality, fertility, and income datasets were made tidy, so that important can be done in a relatively consistent manner. The important feature for the infant mortality dataset is the 'Mortality Rate'.

In [ ]:
# Get the mortality table using the parser.read_mfi helper function.
mortality_df = parser.read_mfi(
    '../data/mfi/mortality/mortality_long.tsv', 
    title="Mortality Rate",
    countries=selected_df.index)    
display(mortality_df)

In [ ]:
# Get the fertility table using the parser.read_mfi helper function.
fertility_df = parser.read_mfi(
    '../data/mfi/fertility/fertility_long.tsv', 
    title="Fertility Rate",
    countries=selected_df.index)    
display(fertility_df)

In [ ]:
# Get the income table using the parser.read_mfi helper function.
income_df = parser.read_mfi(
    '../data/mfi/income/income_long.tsv', 
    title="Income",
    countries=selected_df.index)    
display(income_df)

<a id="import_data_gtd" />

#### Import Terrorism Database ####

In [ ]:
# Import the terrorism database.
crime_df = parser.read_tsv('../data/gtd/gtd.tsv')

# Rename columns.
crime_df.columns = ['Event ID', 'Country ID', 'Country', 'Year', 'Success', 'Attack Type ID', 'Attack Type', 'Killed', 'Wounded']

In [ ]:
# Import the terrorism database.
crime_df = parser.read_tsv('../data/gtd/gtd.tsv')

# Rename columns.
crime_df.columns = ['Event ID', 'Country ID', 'Country', 'Year', 'Success', 'Attack Type ID', 'Attack Type', 'Killed', 'Wounded']

In [ ]:
# Descriptive statistics regarding the entire dataset.
print("Results for the overall dataset:")
display(crime_df.describe(include=np.object))
display(crime_df.drop(labels=['Event ID', 'Country ID'], axis=1).describe())

In [ ]:
# Preparing for queries.
print("Selecting for candidate countries...")
crime_df = crime_df[crime_df['Country ID'].isin(selected_df['ID'])]

print("Mapping country ID to country code...")
id_map = dict(selected_df[['ID', 'Code']].values)
crime_df['Code'] = crime_df['Country ID'].map(id_map)

print("Summing to get casualty total...")
crime_df['Casualties'] = crime_df['Killed'] + crime_df['Wounded']

print("Reorganize columns...")
crime_df = crime_df[['Event ID', 'Code', 'Country', 'Year', 'Attack Type', 'Killed', 'Wounded', 'Casualties', 'Success']]

In [ ]:
display(crime_df)

<a id="import_data_ped" />

#### Import Public Expenditure Data ####

In [ ]:
# Import the public expenditure database.
ped_df = parser.read_tsv('../data/ped/ped.tsv')
display(ped_df)

In [ ]:
print("Rename columns...")
ped_df = ped_df.rename(columns={ 'Entity': 'Country', 'Public Expenditure on Education (percent of GDP)': "%GDP" })

print("Reorder columns...")
ped_df = ped_df[['Code', 'Country', 'Year', '%GDP']]

print("Selecting countries...")
ped_df = ped_df[ped_df['Code'].isin(selected_df['Code'])]

print("Sorting by year...")
ped_df = ped_df.sort_values(by=["Code", "Year"], axis=0)
display(ped_df)

In [ ]:
# Descriptions for selected countries.
display(ped_df.describe(include=np.object))
display(ped_df.describe())
display(ped_df.groupby(['Code']).describe().drop(labels="Year", axis=1))

***

<a id="analysis" />

## Queries ##

The objective of the research project is to evaluate key indicators of stability and growth across several countries in order to estimate Ryder's potential for success in these new markets.

This sample report evaluates the trends in three (3) separate countries as a sampling of what the broader international research has to offer. **The United Kingdom** (GBR), **Japan** (JPN), and **Sweden** (SWE).

The **United States of America** (USA) is included in order to use our current operation region for context.

For each of these countries, we want to answer the following questions:

    1. Is infant mortality improving, stable, or getting worse?
    2. Is income rising, stagnant, or falling?
    3. Does one country or another seem more or less stable than the others, and why do you say this?
    4. What changes do you predict for these countries, and why?



### Infant Mortality ###

For **each country** in `selected_countries`, observations regarding infant mortality per 1,000 live births can be determined with the following steps:

- Summraize rates of infant mortality per 1,000 live births per year, for each country.

- Plot a time-series showing infant mortality trends in each country.

In [ ]:
# Query 1a. Average infant mortality per 1,000 live births per year, by country.
# print(describe_numeric(mortality_df.groupby(['Code']), 'Mortality Rate'))
mortality_stats_year = mortality_df.groupby(['Code']).agg({
    'Year': [
        'count',
        'min',
        'max',
        analyser.spread(),
        analyser.percentile(0),
        analyser.percentile(0.25),
        analyser.percentile(0.5),
        analyser.percentile(0.75),
        analyser.percentile(1),
        analyser.IQR()
    ]
}).dropna(axis=1,how='all')
print(mortality_stats_year)

mortality_stats_rate = mortality_df.groupby(['Code']).agg({
    'Mortality Rate': [
        'min',
        'idxmin',
        'max',
        'idxmax'
    ]
}).dropna(axis=1,how='all')
print(mortality_stats_rate)

mortality_quartiles = mortality_df.groupby(['Code']).agg({ 
    'Mortality Rate': [ 
        analyser.percentile(0), 
        analyser.percentile(0.25),
        analyser.percentile(0.5),
        analyser.percentile(0.75),
        analyser.percentile(1),
        analyser.IQR(),
        analyser.spread()
    ]
})
print(mortality_quartiles)

In [ ]:
# Query 1b. Time-series plot showing infant mortality trends, by country.

df = mortality_df.drop(labels=['original_index'], axis=1)

num_years = df['Year'].nunique()
num_countries = df['Code'].nunique()
year_min = df['Year'].min()
year_max = df['Year'].max()

options = {
    'xlabel': 'year',
    'ylabel': 'infant mortality (per 1,000 live births)',
    'title': f'Infant mortality rate, by country ({year_min} to {year_max})',
    'xlim': (year_min - 2, year_max + 2),
    'ylim': (-2, df['Mortality Rate'].max() * 1.25),
}

# Plot the mortality information.
print(f'Plotting {options["title"]} for each {num_countries} countr(y/ies) across {num_years} year(s)...')
fig, ax = plt.subplots()

for key, grp in df.groupby(['Code']):
    ax.plot('Year', 'Mortality Rate', 'o-', data=grp, label=df[df['Code'] == key]['Country'].unique()[0])

ax.set(**options)

ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(15))

ax.legend(loc='upper right', frameon=False)
ax.grid(True)

### National Net Income per Capita ###

For **each country** in `selected_countries`, observations regarding NNI per Capita (in 2018 $USD) can be determined with the following steps:

- Summarize the NNI per Capita per year, for each country.

- Plot a time-series showing NNI per Capita trends in each country.

In [ ]:
# Query 2a. Summarize the NNI per Capita per year, for each country.
income_stats = income_df.groupby(['Code']).agg({
    'Income': [
        'min',
        'idxmin',
        'max',
        'idxmax'
    ]
}).dropna(axis=1,how='all')
print(income_stats)

income_quartiles = income_df.groupby(['Code']).agg({ 
    'Income': [ 
        analyser.percentile(0), 
        analyser.percentile(0.25),
        analyser.percentile(0.5),
        analyser.percentile(0.75),
        analyser.percentile(1),
        analyser.IQR(),
        analyser.spread()
    ]
})
print(income_quartiles)


In [ ]:
# Query 2b. Plot a time-series showing NNI per Capita trends in each country.
df = income_df.drop(labels=['original_index'], axis=1)

num_years = df['Year'].nunique()
num_countries = df['Code'].nunique()
year_min = df['Year'].min()
year_max = df['Year'].max()

options = {
    'xlabel': 'year',
    'ylabel': 'national net income per capita',
    'title': f'NNI per Capita, by country ({year_min} to {year_max}) in 2018 $USD',
    'xlim': (year_min - 2, year_max + 2),
    'ylim': (-2, df['Income'].max() * 1.25),
}

# Plot the mortality information.
print(f'Plotting {options["title"]} for each {num_countries} countr(y/ies) across {num_years} year(s)...')
fig, ax = plt.subplots()

for key, grp in df.groupby(['Code']):
    ax.plot('Year', 'Income', 'o-', data=grp, label=df[df['Code'] == key]['Country'].unique()[0])

ax.set(**options)

ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(15))
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('${x:,.0f}'))

ax.legend(loc='upper left', frameon=True)
ax.grid(True)

plt.show()

### Crime & Stability ###

For **each country** in `selected_countries`, observations regarding terrorist incidents can be determined with the following steps:

- Summarize the terrorism incidents per year, for each country.

- Plot a time-series showing crime and stability in each country.

In [ ]:
# Query 3a. Summarize the terrorism incidents per year, for each country.
display(crime_df.groupby(['Code']).nunique()[['Event ID', 'Year', 'Attack Type']])

crime_stats = crime_df.groupby(['Code']).agg({
    'Killed': [
        'min',
        'idxmin',
        'max',
        'idxmax',
        'count',
        'sum',
    ],
    'Wounded': [
        'min',
        'idxmin',
        'max',
        'idxmax',
        'sum',
    ],   
    'Casualties': [
        'min',
        'idxmin',
        'max',
        'idxmax',
        'sum',
    ],   
    'Attack Type': [
        'count',
        'max',
        'nunique',
        analyser.mode(), 
    ], 
}).dropna(axis=1,how='all')
display(crime_stats)

crime_quartiles = crime_df.groupby(['Code']).agg({ 
    'Killed': [ 
        analyser.mode(), 
        analyser.percentile(0), 
        analyser.percentile(0.25),
        analyser.percentile(0.5),
        analyser.percentile(0.75),
        analyser.percentile(1),
        analyser.IQR(),
        analyser.spread()
    ],
    'Wounded': [ 
        analyser.mode(), 
        analyser.percentile(0), 
        analyser.percentile(0.25),
        analyser.percentile(0.5),
        analyser.percentile(0.75),
        analyser.percentile(1),
        analyser.IQR(),
        analyser.spread()
    ],
    'Casualties': [
        analyser.mode(), 
        analyser.percentile(0), 
        analyser.percentile(0.25),
        analyser.percentile(0.5),
        analyser.percentile(0.75),
        analyser.percentile(1),
        analyser.IQR(),
        analyser.spread()
    ],   
})
display(crime_quartiles)

In [ ]:
crime_desc = crime_df.drop(labels=['Event ID'], axis=1).groupby(['Code', 'Attack Type']).describe()

print('Country Terrorist Attacks by Year')
display(crime_desc['Year'][['count', 'min', '50%', 'max']])

print('Country Terrorist Attack Casualties')
display(crime_desc[['Killed', 'Wounded']].drop(labels=['25%', '75%', 'mean', 'std'], axis=1, level=1))

print('Country Terrorist Attack Casualties')
display(crime_desc['Casualties'])


In [ ]:
# Query 3b. Plot a time-series showing crime and stability in each country.
df = crime_df.sort_values(by=['Code','Event ID'])
df['Incidents'] = df.groupby(['Code']).cumcount()

num_attack_max = df['Incidents'].max()
num_years = df['Year'].nunique()
num_countries = df['Code'].nunique()
year_min = df['Year'].min()
year_max = df['Year'].max()

# Fill in missing years by padding using the previous year's values.
df['Incidents'] = df['Incidents'].interpolate()

options = {
    'xlabel': 'year',
    'ylabel': 'number of incidents',
    'title': f'Terror Incidents, by country ({year_min} to {year_max})',
    'xlim': (year_min - 1, year_max + 1),
    'ylim': (-2, num_attack_max * 1.25),
}

# Plot the crime information.
print(f'Plotting {options["title"]} for each {num_countries} countr(y/ies) across {num_years} year(s)...')
fig, ax = plt.subplots()

for key, grp in df.groupby(['Code']):
    ax.plot(grp['Year'], grp['Incidents'], '.-', label=df[df['Code'] == key]['Country'].unique()[0])

ax.set(**options)

ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(15))
# ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('${x:,.0f}'))

ax.legend(loc='upper left', frameon=True)
ax.grid(True)

plt.show()


In [ ]:
options = {
    'xlabel': 'year',
    'ylabel': 'number of casualties',
    'title': f'Casualties due to terrorism, by country ({year_min} to {year_max})',
    'xlim': (year_min - 1, year_max + 1),
    'ylim': (0, df['Casualties'].max() * 1.25),
}

# Plot the crime information.
print(f'Plotting {options["title"]} for each {num_countries} countr(y/ies) across {num_years} year(s)...')
fig, ax = plt.subplots()

for key, grp in df.groupby(['Code']):
    ax.plot(grp['Year'], grp['Casualties'], 'o-', label=df[df['Code'] == key]['Country'].unique()[0])

ax.set(**options)

ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(10))
ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(15))
# ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('${x:,.0f}'))

ax.legend(loc='upper left', frameon=True)
ax.grid(True)

plt.show()


<a id="predict" />

### Predictions ###

Predictions 

<a id="predict_usa" />

#### The United States ####

Is it stable?
Is it a good fit?

This serves as a baseline to compare to other countries.

<a id="predict_gbr" />

#### The United Kingdom ####

Is it stable?
Is it a good fit?
How does it compare to the USA?

<a id="predict_swe" />

#### Sweden ####

Is it stable?
Is it a good fit?
How does it compare to the USA?

Other considerations: Language. Would need to hire new staff.

<a id="predict_jpn" />

#### Japan ####

Is it stable?
Is it a good fit?
How does it compare to the USA?

Other considerations: Language. Would need to hire new staff.